<div align='center'>
    <h2>Controle Quântico Ótimo</h2>
    <h4>Utilização do Algoritmo Genético para obtenção dos valores de controle</h4>
</div>

<div align='justify'>
    <p>Para que haja um controle ótimo de uma partícula, supondo que esta esteja em um estado inicial, é necessário formas de controle suficientemente eficientes já que a particula é a função de onda da partícula é definida em função do tempo. Para que isso seja possível utiliza-se a eficiência do algoritmo genético que pode usar ou não utilizar-se do elitismo juntamente com o controle preditivo, MPC. </p>
    <p>Abaixo são revelados os parâmetros para esse experimento, do qual podem ser ajustados:</p>
</div>

In [ ]:
# Bilbiotecas para auxílio na programação matemática
import math, sys 
import numpy as np
import sympy as sp
import cmath

from scipy import sparse # Produção das diagonais das matrizes
from scipy.sparse import diags 

# Plotagem 2D e 3D
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm


from os import path # Suficiente para manipulação de arquivos
    
# Para solução exata
from scipy.special import hermite
from math import factorial

%matplotlib inline
count = 0

# Para otimização dos sistemas
from scipy import optimize
from random import randint, uniform, random

In [ ]:
## CONSTANTES PARA O CONTROLADOR

# Considerações: constante de Planck verdadeira: 1, massa: 1

TEMPO_ANALISE = 10 
QUANTIDADE_PONTOS_AMOSTRAGEM = 50
PASSO_RUNGE_KUTTA = TEMPO_ANALISE/QUANTIDADE_PONTOS_AMOSTRAGEM


MIN_HORIZONTE = 3 # Horizonte mínimo da análise
MAX_HORIZONTE = 40 # Horizonte máximo da análise

ONDA_DESEJADA = [[complex(1/np.sqrt(2),0.0)],[complex(1/np.sqrt(2),0.0)]] # [[c1];[c2]] = [[0.707];[0.707]]

In [ ]:
## CONSTANTES PARA O ALGORITMO GENÉTICO

ELITISMO = True
TAM_POP = 50
MIN = 0
MAX = 5
MUTACAO = 0.10
CROSS = 0.70
GERACOES = 100